In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from IPython import display
torch.cuda.is_available()

True

In [4]:
def devide(x,y):
    return x/y if y!=0 else 'Non'

devide(10,2)

5.0

In [ ]:
#بسم الله الرحمن الرحيم

In [ ]:
# حملنا شبكة اليولو النسخة الخامسة
!git clone https://github.com/ultralytics/yolov5

In [ ]:
#ندخل لملف yolov5 
!cd yolov5

#نحمل المتطلبات المسبقة
!pip install -r requirements.txt

In [108]:
!cd yolov5

#هنا تمرين الشبكة على بيانات المشروع، حجم الصور 416*416 ولكل حزمة 16 صورة، 

#مرنّا الشبكة ابتداء وأمررناها على جميع البيانات 50 مرة، وحفظنا الأوزان في المسار الموضح أدناه
#yolov5/runs/train/exp5/weights/last.p

#وبعد برمجة الهيكل الأساسي للمشروع على ذاك النموذج المبدئي، قمنا بتدريبه مئتي مرة إضافية لرفع الدقة
python train.py --img 416 --batch 16 --epochs 200 --data sign.yaml --weights runs/train/exp5/weights/last.pt --workers 2


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
#هنا نستدعي نموذجنا الممرن من المسار الموضح أدناه

#yolov5/runs/train/exp7/weights/best.pt


# علما بأنا قمنا برفع الأوزان التي مرنّاها فعلى من يحملها أن يستبدل المسار أدناه بمسار الأوزان في جهازه

model = torch.hub.load('ultralytics/yolov5', 'custom', path = 'yolov5/runs/train/exp7/weights/best.pt', force_reload = True)


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Master/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.2 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 970, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7096429 parameters, 0 gradients, 16.0 GFLOPs
Adding AutoShape... 


In [4]:
model.names

{0: 'ain',
 1: 'al',
 2: 'aleff',
 3: 'bb',
 4: 'dal',
 5: 'dha',
 6: 'dhad',
 7: 'fa',
 8: 'gaaf',
 9: 'ghain',
 10: 'ha',
 11: 'haa',
 12: 'jeem',
 13: 'kaaf',
 14: 'khaa',
 15: 'la',
 16: 'laam',
 17: 'meem',
 18: 'nun',
 19: 'ra',
 20: 'saad',
 21: 'seen',
 22: 'sheen',
 23: 'ta',
 24: 'taa',
 25: 'thaa',
 26: 'thal',
 27: 'toot',
 28: 'waw',
 29: 'ya',
 30: 'yaa',
 31: 'zay'}

In [5]:
class_to_letter2 = {0: 'ع',
 1: 'لﺍ',
 2: 'ا',
 3: 'ب',
 4: 'د',
 5: 'ظ',
 6: 'ض',
 7: 'ف',
 8: 'ق',
 9: 'غ',
 10: 'ه',
 11: 'ح',
 12: 'ج',
 13: 'ك',
 14: 'خ',
 15: 'فﺬﺣ',
 16: 'ل',
 17: 'م',
 18: 'ن',
 19: 'ر',
 20: 'ص',
 21: 'س',
 22: 'ش',
 23: 'ت',
 24: 'ط',
 25: 'ث',
 26: 'ذ',
 27: 'ة',
 28: 'و',
 29: 'ﺔﻓﺎﺴﻤﻟﺍ', 
 30: 'ي',
 31: 'ز'}

In [6]:
for idx,name in model.names.items():
    model.names[idx] = class_to_letter2[idx]
    
    #هنا قمنا بمقابلة كل اسم إنجليزي للحرف العربي، بنظيره من الرموز العربية

In [7]:
model.names

{0: 'ع',
 1: 'لﺍ',
 2: 'ا',
 3: 'ب',
 4: 'د',
 5: 'ظ',
 6: 'ض',
 7: 'ف',
 8: 'ق',
 9: 'غ',
 10: 'ه',
 11: 'ح',
 12: 'ج',
 13: 'ك',
 14: 'خ',
 15: 'فﺬﺣ',
 16: 'ل',
 17: 'م',
 18: 'ن',
 19: 'ر',
 20: 'ص',
 21: 'س',
 22: 'ش',
 23: 'ت',
 24: 'ط',
 25: 'ث',
 26: 'ذ',
 27: 'ة',
 28: 'و',
 29: 'ﺔﻓﺎﺴﻤﻟﺍ',
 30: 'ي',
 31: 'ز'}

In [15]:
model.conf = 0.82 #أدنى درجة ثقة مقبولة ليعدّ الرصد معتبرًا

In [9]:
class_to_letter = {0: 'ع',
 1: 'ى',
 2: 'ا',
 3: 'ب',
 4: 'د',
 5: 'ظ',
 6: 'ض',
 7: 'ف',
 8: 'ق',
 9: 'غ',
 10: 'ه',
 11: 'ح',
 12: 'ج',
 13: 'ك',
 14: 'خ',
 15: 'ﻻ',
 16: 'ل',
 17: 'م',
 18: 'ن',
 19: 'ر',
 20: 'ص',
 21: 'س',
 22: 'ش',
 23: 'ت',
 24: 'ط',
 25: 'ث',
 26: 'ذ',
 27: 'ة',
 28: 'و',
 29: ' ',
 30: 'ي',
 31: 'ز'}

In [16]:



text = ""
cap = cv2.VideoCapture(0) #البث الحي
cond = None 

while cap.isOpened(): #اللوب الأساسي
 
    ret, frame = cap.read()

    results = model(frame)
    cv2.imshow("Sign Language Translator", results.render()[0])

    if results.pred[0].shape[0] and results.pred[0].shape[1]: #لا يدخل إلا إن رصد رمزًا
        current_letter = class_to_letter[int(results.pred[0][0][-1])] #انتزاع الحرف من النتيجة
    
    
    
         #حشد الحروف وطباعتها مباشرة
        if (current_letter != text[-1:]):
            
            if (current_letter != 'ﻻ'):
                text += current_letter
                display.clear_output(wait=True)

                print(text,end="\r")

    
    
        #خاصية حذف الحرف أثناء الترجمة المباشرة
        if (current_letter == 'ﻻ'):
            if cond is None or time.time() - cond >= 0.5:
                text = text[:-1]
                cond = time.time()
                display.clear_output(wait=True)
                print(text,end="\r")


            

        #qللخروج اضغط حرف الـ
    if (cv2.waitKey(3) & 0xFF == ord('q')) :
        break

cap.release()
cv2.destroyAllWindows()

